# __Econometric Game__: Effect of electoral accountability on corruption?
   
### Team 3: Katheryn Ding, Amber Wei, Max Ye



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import matplotlib.ticker as ticker


## Set up 

In [7]:
corruption_df=pd.read_stata("corruptiondata.dta")
corruption_df.columns

Index(['uf', 'nsorteio', 'totrecursos', 'tot_os', 'pop', 'purb',
       'p_secundario', 'cod_ibge6', 'pib_capita_02', 'op_01_04',
       ...
       'uf_d18', 'uf_d19', 'uf_d20', 'uf_d21', 'uf_d22', 'uf_d23', 'uf_d24',
       'uf_d25', 'uf_d26', 'esample2'],
      dtype='object', length=116)

In [ ]:
=

,Mayoral,Municipal,P&J,dum
0,pref_idade_tse,lpop,ENEP2020,uf_d*
1,pref_masc,purb,ENLP2020,sorteio*
2,party_d*,p_secundario,p_cad_pref,None
3,pref_escola,lpib02,None,None
4,winmargin2020,gini_ipea,None,None
5,exp_prefeito,None,None,None


## Lasso 

## Random Forest

## Neural Network

## Gradient Boosting


## Analysis 

## Conclusion